# KMEANS

### 18/05/20

## Importing Libraries

In [132]:
# Import all libraries needed for the tutorial

# General syntax to import specific functions in a library: 
##from (library) import (specific library function)
from pandas import DataFrame, read_csv

# General syntax to import a library but no functions: 
##import (library) as (give the library a nickname/alias)
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

# Enable inline plotting
%matplotlib inline

import scipy
import numpy as np
#import scipy.signal as signal
from scipy.signal import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize

import sys
#sys.path.insert(0, '/Users/SYL21/External_Drive/SUPAERO/PIR/AdabandFlt')
#sys.path.insert(0, '/Users/louiseplacidet/Desktop/PIR/GITPIR/GIT_29_04/PIR/AdabandFlt')
#from AdaBandFlt import *
#from V2AdaBandFlt import *
from V3AdaBandFlt import *

%matplotlib tk

## Loading Data

In [47]:
#load data
# file path of csv file
#Location = r'/Users/33614/ExternalDrive/SUPAERO/PIR_2A/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/SYL21/D_Drive/SUPAERO/PIR_2A/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/louiseplacidet/Desktop/PIR/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/SYL21/External_Drive/SUPAERO/PIR/Data/Wetransfer_data/E18KABaseline_BcutV2groundAll.txt'
#Location = r'/Users/louiseplacidet/Desktop/PIR/Data/new_spike_data/newdata/E18KABaseline_BcutV2groundAll.txt'

Location = r'/Users/louiseplacidet/Desktop/PIR/Data/new_new_spike_data/539W6cbaseRaw.txt'


# create dataframe
df = pd.read_csv(Location, sep='\t',skiprows=[0,1,3] , index_col='%t           ')
df

/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,El 41,El 42,El 44,El 54,El 55,El 56,El 58,El 86,El 15,El 51
%t,,,,,,,,,,
0.00,0.92,-1.46,2.32,-0.61,0.06,-1.04,8.06,-0.24,0.73,-0.55
0.02,0.49,-1.53,1.59,-0.31,0.73,-1.22,6.29,-0.85,0.92,-1.40
0.04,0.43,-1.34,0.92,-0.37,1.16,-1.10,4.15,-0.92,0.85,-2.08
0.06,0.85,-0.85,0.37,-0.85,1.04,-0.92,2.08,-0.73,0.61,-2.44
0.08,1.46,-0.37,-0.12,-1.46,0.31,-0.73,0.55,-0.55,0.18,-2.38
...,...,...,...,...,...,...,...,...,...,...
299999.90,1.34,0.73,4.21,-1.95,-4.76,-1.28,-6.96,-3.60,-0.31,2.50
299999.92,1.59,0.49,4.09,-2.44,-3.42,-1.34,-6.04,-4.09,0.00,2.93
299999.94,2.08,0.37,3.66,-3.48,-1.95,-1.53,-5.13,-4.64,0.18,2.93


 # Filtering the signal for all electrodes

### Global Parameters

In [5]:
## Parameters to choose from: 

size_of_data = 1000000
fs = 50000

method_align = 'indice_1er_depass'
time_before = 0.0015
time_after = 0.0015

n_electrode = 4

### Filtering Functions

In [6]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

### Filtering the electrode signals...

In [9]:
lowcut = 100.0
highcut = 5000.0

filtered_signal = []

for electrode in range(n_electrode):
    before_filter = df.iloc[:size_of_data,electrode]
    
    y = butter_bandpass_filter(before_filter, lowcut, highcut, fs, order=6)
    
    filtered_signal.append(y)
    
filtereddf_not_transposed = pd.DataFrame(filtered_signal)

    
filtereddf = filtereddf_not_transposed.transpose()
filtereddf.index = df.index[:size_of_data]
filtereddf.columns = df.columns[:n_electrode]

filtereddf

,El 41,El 42,El 44,El 54
%t,,,,
0.00,-0.655738,0.844880,0.203710,0.969816
0.02,-0.527641,0.924851,-0.546749,0.604473
0.04,-0.395290,1.032649,-1.233368,0.232775
0.06,-0.268779,1.204451,-1.803244,-0.115492
0.08,-0.179256,1.486950,-2.223309,-0.358307
...,...,...,...,...
19999.90,0.302640,-0.729945,0.729187,0.272311
19999.92,-0.028468,-0.692800,0.402893,0.885210
19999.94,-0.186762,-0.519373,0.196920,1.023643


# Detecting and Aligning the Spikes

### Setting Up Thresholds adapted to Noise Levels for each electrode

In [10]:
noise_levels_all_electrodes = []

for electrode in range(n_electrode):
    noise_levels = init_noise_levels(filtereddf.iloc[:size_of_data, electrode], fs, 
                      noise_window_size = 0.01,
                      required_valid_windows = 20,
                      old_noise_level_propagation = 0.8, 
                      test_level = 5,
                      estimator_type = "RMS",
                      percentile_value = 25)
    noise_levels_all_electrodes.append(noise_levels)


In [12]:
# Plotting the noise levels for all electrodes
plt.figure()
for electrode in range(n_electrode):
    plt.plot(noise_levels_all_electrodes[electrode])
plt.grid(True)
plt.xlabel('Time')
plt.ylabel('Noise Amplitude [µV]')
plt.title('Noise Levels')

Text(0.5, 1.0, 'Noise Levels')

### Detecting Spikes For Each Electrode

In [72]:
list_spike_info = []

for electrode in range(n_electrode):
    spike_info = find_spikes(filtereddf.iloc[:size_of_data, electrode], noise_levels_all_electrodes[electrode], fs, 
                          window_size = 0.001, 
                          noise_window_size = 0.01,
                          threshold_factor = 3.0,
                          maxseparation = 0.002)
    spike_info['Electrode num'] = electrode
    #Normalizing the amplitudes in a new column
    amplitude_norm = np.linalg.norm(spike_info['Delta_amplitudes'])
    spike_info['Delta_amp_norm'] = spike_info['Delta_amplitudes']/amplitude_norm
    list_spike_info.append(spike_info)

In [73]:
list_spike_info

[    indice_min  indice_1er_depass indice_max_gauche  indice_max_droite  \
 0        52722              52721               nan              52800   
 1       121806             121804               nan             121830   
 2       145788             145787               nan             145843   
 3       155751             155750               nan             155830   
 4       165100             165098               nan             165156   
 5       312104             312101               nan             312183   
 6       370196             370195               nan             370258   
 7       379070             379069               nan             379126   
 8       477704             477704               nan             477771   
 9       521397             521397               nan             521482   
 10      609938             609937               nan             609978   
 11      648772             648771               nan             648790   
 12      688514          

### Recording the Spikes For Each Electrode

In [49]:
list_spike_data = []

for electrode in range(n_electrode):
    spike_data = record_spikes(filtereddf.iloc[:size_of_data, electrode], fs, list_spike_info[electrode],
                           method_align,
                           t_before = time_before, 
                           t_after = time_after)
    spike_data = spike_data.drop(columns=[0])
    list_spike_data.append(spike_data)

(20, 151)
(27, 151)
(20, 151)
(26, 151)


In [90]:
list_spike_data;

[           1         2         3         4         5         6         7   \
 0    2.506359 -3.742077  4.292483  1.264332  1.022628 -1.469511  0.747333   
 1    3.640852 -3.009869  3.372459  0.060058  0.320394 -0.120403  0.980394   
 2    4.407635 -2.117797  2.388609 -0.975903 -0.444605  1.125369  1.166655   
 3    4.714776 -1.077103  1.467253 -1.747182 -1.173047  2.142577  1.290294   
 4    4.570209  0.068284  0.700083 -2.226175 -1.799870  2.857106  1.353612   
 ..        ...       ...       ...       ...       ...       ...       ...   
 146  0.081180  2.158286 -1.317528  1.964246 -2.415784 -1.137577  2.525267   
 147  0.462321  2.302786 -1.480212  1.423667 -2.802496 -1.766126  1.865172   
 148  1.240974  2.652295 -1.392558  1.263563 -2.913669 -2.182174  1.419365   
 149  2.344133  3.160435 -1.054048  1.649268 -2.626920 -2.205063  1.240851   
 150  3.623066  3.711983 -0.489896  2.599201 -1.906167 -1.721908  1.339109   
 
            8         9         10        11        12        

In [85]:
list_spike_data_normalized = []

for electrode in range(n_electrode):
    spike_data_normalized = record_spikes(filtereddf.iloc[:size_of_data, electrode], fs, list_spike_info[electrode],
                           method_align,
                           t_before = time_before, 
                           t_after = time_after)
    spike_data_normalized = spike_data_normalized.drop(columns=[0])
    for i in range(len(spike_data_normalized.columns)):
        amp_norm = np.linalg.norm(spike_data_normalized[i+1])
        spike_data_normalized[i+1] = spike_data_normalized[i+1]/amp_norm
    list_spike_data_normalized.append(spike_data_normalized)



(20, 151)
(27, 151)
(20, 151)
(26, 151)


In [92]:
list_spike_data_normalized;

[           1         2         3         4         5         6         7   \
 0    0.062658 -0.097295  0.113404  0.044577  0.027054 -0.048089  0.020329   
 1    0.091019 -0.078257  0.089098  0.002118  0.008476 -0.003940  0.026669   
 2    0.110189 -0.055063  0.063105 -0.034408 -0.011762  0.036827  0.031736   
 3    0.117867 -0.028005  0.038764 -0.061601 -0.031033  0.070115  0.035099   
 4    0.114253  0.001775  0.018496 -0.078489 -0.047616  0.093498  0.036821   
 ..        ...       ...       ...       ...       ...       ...       ...   
 146  0.002029  0.056116 -0.034808  0.069254 -0.063910 -0.037227  0.068693   
 147  0.011558  0.059873 -0.039106  0.050195 -0.074141 -0.057796  0.050737   
 148  0.031024  0.068960 -0.036790  0.044550 -0.077082 -0.071411  0.038610   
 149  0.058602  0.082172 -0.027847  0.058149 -0.069496 -0.072160  0.033754   
 150  0.090575  0.096512 -0.012943  0.091641 -0.050428 -0.056349  0.036427   
 
            8         9         10        11        12        

In [51]:
list_spike_data_oneline = []

for electrode in range(n_electrode):
    spike_data_oneline = record_spikes_oneline(filtereddf.iloc[:size_of_data, electrode], fs, list_spike_info[electrode],
                                           method_align,
                                           t_before = time_before,
                                           t_after = time_after)
    list_spike_data_oneline.append(spike_data_oneline)

### Plotting the Spikes For Each Electrode

In [53]:
#Plotting the spikes on top of original signal

for electrode in range(n_electrode):
    plt.figure()
    plt.plot(filtereddf.index[:size_of_data], filtereddf.iloc[:size_of_data,electrode], color = 'blue')
    plt.plot(filtereddf.index[:size_of_data], list_spike_data_oneline[electrode], color = 'red')
    plt.title('Filtered Signal with Detected Spikes with RMS for electrode:'+str(filtereddf.columns[electrode]))
    plt.xlabel('Time Windows')
    plt.ylabel('Amplitude [µV]')
    plt.legend()
    plt.grid(True)

No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
/opt/anaconda3/lib/python3.7/tkinter/__init__.py:749: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)
/opt/anaconda3/lib/python3.7/tkinter/__init__.py:1705: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  return self.func(*args)


In [97]:
from random import randint

def print_spikes_adapted(spike_data,
                 t_before_alignement = 0,
                 first_spike = 1,
                 last_spike = -1,
                 fs = 25000,
                 randomize = False,
                 nb_spike = 20,
                 y_lim_min = -0.5,
                 y_lim_max = 0.5):
    
    if randomize == True:        
        kept = []
        m = len(spike_data.values[0])
        if m <= nb_spike:
            kept = [i for i in range(m)]
        else:      
            i = 0  
            while i < nb_spike:
                r = randint(0,m-1)
                if (r in kept) == False:
                    kept.append(r)
                    i += 1
        
        x = spike_data.iloc[:,kept].values
        
    else:
        x = spike_data.iloc[:,first_spike:last_spike]
        
    figure = plt.figure()
    t_b = int(np.round(fs*(t_before_alignement)))
    axes = figure.add_subplot(1, 1, 1)
    axes.plot((spike_data.index-t_b)*1000/fs, x)
    axes.set_xlabel('Time in ms')
    axes.set_ylim(y_lim_min , y_lim_max)
    axes.grid()

In [99]:
# Plotting aligned spikes for each electrode
for electrode in range(n_electrode):
    print_spikes_adapted(list_spike_data_normalized[electrode],
             t_before_alignement = time_before,
             first_spike = 1,
             last_spike = 50,
             fs = fs)
    plt.title("Spikes aligned on "+str(method_align)+" for electrode "+str(filtereddf.columns[electrode]))

# Bilan PCA + KMEANS

### Merging spike_data and spike_info from all electrodes

In [25]:
spike_data_all_electrodes = pd.concat([list_spike_data[electrode] for electrode in range(n_electrode)], axis=1)
spike_data_all_electrodes.columns = [i+1 for i in range(len(spike_data_all_electrodes.columns))]
spike_data_all_electrodes

,1,2,3,4,5,6,7,8,9,10,...,80,81,82,83,84,85,86,87,88,89
0,2.506359,-3.742077,4.292483,1.264332,1.022628,-1.469511,0.747333,5.560564,1.778498,-4.015044,...,-0.641910,1.070805,-3.342572,0.208475,-5.801737,-2.369335,1.345531,1.127725,0.511525,-2.209425
1,3.640852,-3.009869,3.372459,0.060058,0.320394,-0.120403,0.980394,4.774651,1.184161,-3.082622,...,-0.445390,0.933783,-3.012648,1.067454,-6.096784,-1.472113,0.993708,1.875437,0.599718,-2.510246
2,4.407635,-2.117797,2.388609,-0.975903,-0.444605,1.125369,1.166655,3.614611,0.466642,-1.849261,...,-0.432742,0.551494,-2.515348,1.831904,-6.082740,-0.669545,0.696922,2.499759,0.520177,-2.617527
3,4.714776,-1.077103,1.467253,-1.747182,-1.173047,2.142577,1.290294,2.181049,-0.207246,-0.532872,...,-0.614138,0.026806,-1.929151,2.275267,-5.727671,-0.093733,0.502799,2.970784,0.303325,-2.430474
4,4.570209,0.068284,0.700083,-2.226175,-1.799870,2.857106,1.353612,0.609682,-0.677275,0.639585,...,-0.933740,-0.514056,-1.357088,2.272593,-5.030717,0.199872,0.414969,3.274994,0.029405,-1.911451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,0.081180,2.158286,-1.317528,1.964246,-2.415784,-1.137577,2.525267,-2.038203,6.629625,-5.112417,...,-4.323767,1.727750,-1.128461,1.226375,1.791068,3.766525,1.943728,-2.787651,-3.818057,-2.906062
147,0.462321,2.302786,-1.480212,1.423667,-2.802496,-1.766126,1.865172,-1.707528,6.484901,-5.064729,...,-4.123414,2.054588,-0.662815,1.447434,1.485181,3.231596,3.241027,-2.433324,-3.792061,-2.622004
148,1.240974,2.652295,-1.392558,1.263563,-2.913669,-2.182174,1.419365,-1.068935,6.347552,-4.678602,...,-3.546272,2.302031,-0.360055,1.654308,1.081556,2.435730,4.621145,-1.552895,-3.147343,-2.639121
149,2.344133,3.160435,-1.054048,1.649268,-2.626920,-2.205063,1.240851,-0.234796,6.181344,-3.976821,...,-2.729394,2.414009,-0.310007,1.822646,0.743528,1.529301,5.842984,-0.308053,-2.067521,-2.954162


In [105]:
list_spike_data_normalized

spike_data_norm_all_electrodes = pd.concat([list_spike_data_normalized[electrode] for electrode in range(n_electrode)], axis=1)
spike_data_norm_all_electrodes.columns = [i+1 for i in range(len(spike_data_norm_all_electrodes.columns))]
spike_data_norm_all_electrodes


,1,2,3,4,5,6,7,8,9,10,...,80,81,82,83,84,85,86,87,88,89
0,0.062658,-0.097295,0.113404,0.044577,0.027054,-0.048089,0.020329,0.144061,0.044264,-0.132052,...,-0.014745,0.035569,-0.098881,0.007278,-0.165748,-0.066615,0.036970,0.038258,0.015159,-0.055985
1,0.091019,-0.078257,0.089098,0.002118,0.008476,-0.003940,0.026669,0.123699,0.029472,-0.101385,...,-0.010231,0.031017,-0.089121,0.037267,-0.174177,-0.041389,0.027303,0.063625,0.017772,-0.063608
2,0.110189,-0.055063,0.063105,-0.034408,-0.011762,0.036827,0.031736,0.093646,0.011614,-0.060821,...,-0.009940,0.018319,-0.074410,0.063956,-0.173776,-0.018824,0.019149,0.084805,0.015415,-0.066326
3,0.117867,-0.028005,0.038764,-0.061601,-0.031033,0.070115,0.035099,0.056506,-0.005158,-0.017526,...,-0.014107,0.000890,-0.057069,0.079435,-0.163632,-0.002635,0.013815,0.100785,0.008989,-0.061586
4,0.114253,0.001775,0.018496,-0.078489,-0.047616,0.093498,0.036821,0.015795,-0.016856,0.021035,...,-0.021448,-0.017075,-0.040146,0.079341,-0.143721,0.005619,0.011402,0.111105,0.000871,-0.048435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,0.002029,0.056116,-0.034808,0.069254,-0.063910,-0.037227,0.068693,-0.052805,0.165002,-0.168143,...,-0.099318,0.057390,-0.033383,0.042815,0.051168,0.105897,0.053407,-0.094572,-0.113147,-0.073637
147,0.011558,0.059873,-0.039106,0.050195,-0.074141,-0.057796,0.050737,-0.044238,0.161400,-0.166575,...,-0.094716,0.068247,-0.019608,0.050533,0.042430,0.090857,0.089052,-0.082551,-0.112377,-0.066439
148,0.031024,0.068960,-0.036790,0.044550,-0.077082,-0.071411,0.038610,-0.027693,0.157982,-0.153876,...,-0.081459,0.076466,-0.010651,0.057756,0.030899,0.068481,0.126972,-0.052682,-0.093270,-0.066873
149,0.058602,0.082172,-0.027847,0.058149,-0.069496,-0.072160,0.033754,-0.006083,0.153845,-0.130795,...,-0.062695,0.080186,-0.009171,0.063633,0.021242,0.042997,0.160544,-0.010451,-0.061270,-0.074856


In [26]:
spike_info_all_electrodes = list_spike_info[0].append([list_spike_info[electrode+1] for electrode in range(n_electrode-1)])
spike_info_all_electrodes.index = [i+1 for i in range(len(spike_info_all_electrodes))]
spike_info_all_electrodes

,indice_min,indice_1er_depass,indice_max_gauche,indice_max_droite,Delta_amplitudes,Electrode num
1,52722,52721,nan,52800,14.125634,0
2,121806,121804,nan,121830,15.508620,0
3,145788,145787,nan,145843,15.139943,0
4,155751,155750,nan,155830,15.771986,0
5,165100,165098,nan,165156,14.374165,0
...,...,...,...,...,...,...
85,809120,809118,nan,809168,13.631384,3
86,810616,810614,nan,810690,14.403546,3
87,812015,812012,nan,812108,15.786182,3
88,882454,882453,nan,882470,13.620426,3


## PCA and KMEANS on spikes

In [122]:
def PCA_and_KMEANS_spikes_electrodes_combined(spike_data_norm_all_electrodes, spike_info_all_electrodes, nb_PCA_components=3, nb_clusters=3):
    
    ## PCA
    pca_data = np.array(spike_data_norm_all_electrodes.values).transpose()
    pca = PCA(n_components=nb_PCA_components)
    pca.fit(pca_data)
    PCA_X = pca.transform(pca_data)
    
    ## KMEANS
    kmeans = KMeans(n_clusters=nb_clusters, random_state=0).fit(PCA_X)

    
    ## Ajout du label des clusters dans spike info
    spike_info_all_electrodes['cluster_label'] = kmeans.labels_
    
    return PCA_X, kmeans, spike_info_all_electrodes
    

## Plotting the PCA

In [110]:
## Fonction qui plot la PCA

def PCA_plot(PCA_X):
    
    fig = plt.figure(4,figsize=(4,3))
    plt.clf()
    ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
    plt.cla()
  
    ax.scatter(PCA_X[:, 0], PCA_X[:, 1], PCA_X[:, 2], cmap=plt.cm.nipy_spectral,
           edgecolor='k')

    ax.w_xaxis.set_ticklabels([])
    ax.w_yaxis.set_ticklabels([])
    ax.w_zaxis.set_ticklabels([])

    plt.show()

In [111]:
## Importance des premiers coefficients de la PCA selon les différents indices d'alignement

##ATTENTION! CETTE FONCTION N'EST APPLICABLE QU'À UNE SEULE ÉLECTRODE

def print_pca_explained_variance_ratio(signal, fs, spike_infos):
    legend = ['indice_max',
     'indice_min',
     'indice_depass_positif',
     'indice_depass_negatif',
     'indice_1er_depass',
     'indice_zero_central']
    explained_variance_ratio = []
    
    for method in legend:
        spike_data = record_spikes(signal, fs, spike_infos,
                  method,
                  t_before = 0.001,
                  t_after = 0.002)
        pca_data = np.array(spike_data.iloc[:,1:].values).transpose()
        pca = PCA(n_components=10)
        pca.fit(pca_data)
        X = pca.transform(pca_data)
        explained_variance_ratio.append(pca.explained_variance_ratio_)
        
    plt.figure()
    plt.plot(np.transpose(explained_variance_ratio))
    plt.legend(legend)

## Plotting the KMEANS

In [112]:
## Fonction qui plot le KMEANS

def print_clusters_3d(labels, PCA_X):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    nb_clusters = max(labels) + 1

    a = [i for i in range(len(labels))]
    b = np.transpose([a,list(labels)])

    for nb in range(nb_clusters):
        legend = 'Cluster n°'+str(nb)
        data = PCA_X[[x for x,y in b if y==nb],:]
        ax.scatter(data[:,0], data[:,1], data[:,2],label=legend)
    data = PCA_X[[x for x,y in b if y==-1],:]
    ax.scatter(data[:,0], data[:,1], data[:,2], c='black',label='Noise cluster')    
    
    ax.set_title('Nombre de cluster(s) :' + str(nb_clusters))
    ax.legend()
    plt.show()
    
#print_clusters_3d(kmeans.labels_, X)

## Plotting the spikes from the different clusters after KMEANS

In [164]:
## Fonction qui plot les spikes dans les clusters du KMEANS

def print_spikes_clusterized(spike_data_norm_all_electrodes,
                             labels,
                             t_before_alignement = 0,
                             nb_spike = 20,
                             y_lim_min = -50,
                             y_lim_max = 60,
                             fs = 25000):
    
    nb_clusters = max(labels) + 1
    
    if (-1 in labels) == True:
        nb_clusters_ = nb_clusters + 1
    else:
        nb_clusters_ = nb_clusters
    
    nb_line = nb_clusters_//2
    if nb_clusters_%2 != 0:
        nb_line += 1
    
    #spike_data.iloc[:,first_spike:last_spike].plot()
    t_b = int(np.round(fs*(t_before_alignement)))
    y = (spike_data_norm_all_electrodes.index-t_b)*1000/fs
        
    a = [i for i in range(len(labels))]
    b = np.transpose([a,list(labels)])
    
    figure = plt.figure()
    plt.gcf().subplots_adjust(left = 0.1, bottom = 0.1, right = 0.9, top = 0.9, wspace = 0.2, hspace = 0.5)
    
    for nb in range(nb_clusters):
        
        data = spike_data_norm_all_electrodes.iloc[:,[x for x,z in b if z==nb]]
        m = len(data.values[0])
        nb_spikes_in_cluster = len(data.columns)
        
        
        kept = []
        
        if m <= nb_spike:
            kept = [i for i in range(m)]
        else:      
            i = 0  
            while i < nb_spike:
                r = randint(0,m-1)
                if (r in kept) == False:
                    kept.append(r)
                    i += 1
        
        x = data.iloc[:,kept].values
        axes = figure.add_subplot(nb_line, 2, nb+1)
        axes.plot(y, x)
        axes.set_xlabel('Time in ms')
        axes.set_title('Cluster numero ' + str(nb))
        axes.set_ylim(y_lim_min , y_lim_max)
        axes.grid(True)
        axes.legend([str(nb_spikes_in_cluster)+' spikes'])
        
    if (-1 in labels) == True:
        data = spike_data_norm_all_electrodes.iloc[:,[x for x,y in b if y==-1]]
        m = len(data.values[0])
        nb_spikes_in_cluster = len(data.columns)

        
        kept = []
        
        if m <= nb_spike:
            kept = [i for i in range(m)]
        else:      
            i = 0  
            while i < nb_spike:
                r = randint(0,m-1)
                if (r in kept) == False:
                    kept.append(r)
                    i += 1
        
        x = data.iloc[:,kept].values
        
        axes = figure.add_subplot(nb_line, 2, nb+2)
        axes.plot(y, x)
        axes.set_xlabel('Time in ms')
        axes.set_title('Cluster de bruit')
        axes.set_ylim(y_lim_min , y_lim_max)
        axes.grid(True)
        axes.legend([str(nb_spikes_in_cluster)+' spikes'])
        

#print_spikes_clusterized(spike_data,
#                             kmeans.labels_,
#                             t_before_alignement = 0.0015,
#                             nb_spike = 20,
#                             y_lim_min = -50,
#                             y_lim_max = 60,
#                             fs = 25000)

## Printing the spikes from clusters on original signal

In [114]:
## Fonction qui affiche les spikes des différents clusters sur échelle temporelle

def print_spikes_clusterized_oneline(filtereddf,
                                     updated_spike_info_all_electrodes,
                                     align_method = 'indice_zero_central',
                                     t_before = 0.001,
                                     t_after = 0.002,
                                     fs = 25000,
                                     y_lim_min = -50,
                                     y_lim_max = 60,
                                     separate_plot = False):
    
    electrodes = updated_spike_info_all_electrodes['Electrode num'].values
    nb_electrodes = max(electrodes) + 1
    
    for electrode in range(nb_electrodes):
        
        signal = filtereddf.iloc[:,electrode]
    
        labels = updated_spike_info_all_electrodes['cluster_label'].values
        nb_clusters = max(labels) + 1

        if (-1 in labels) == True:
            nb_clusters_ = nb_clusters + 2
        else:
            nb_clusters_ = nb_clusters + 1

        nb_line = nb_clusters_//2
        if nb_clusters_%2 != 0:
            nb_line += 1

        ghost_array = np.array(['NaN' for x in range(len(signal))])
        ghost_array = ghost_array.astype(float)
        spike_data_clusterized_oneline = []
        legend = ['Signald\'origine']

        figure = plt.figure()
        plt.title(str(filtereddf.columns[electrode]))
        plt.gcf().subplots_adjust(left = 0.1, bottom = 0.1, right = 0.9, top = 0.9, wspace = 0.2, hspace = 0.5)
        for nb in range(nb_clusters):

            spike_data_onelinee = record_spikes_oneline(signal,
                                  fs,
                                  updated_spike_info_all_electrodes.loc[(updated_spike_info_all_electrodes['cluster_label'] == nb) & (updated_spike_info_all_electrodes['Electrode num'] == electrode)],
                                  align_method,
                                  t_before = t_before,
                                  t_after = t_after)
            spike_data_onelinee = np.resize(spike_data_onelinee.values,len(spike_data_onelinee.values))
            spike_data_clusterized_oneline.append(spike_data_onelinee)


            axes = figure.add_subplot(nb_line, 2, nb+1)
            axes.plot(signal.index, signal.values)
            for ghost in range(nb):
                axes.plot(signal.index, ghost_array)
            axes.plot(signal.index, list_spike_data_oneline[electrode])
            axes.set_xlabel('Time in ms')
            axes.set_title('Cluster n°' + str(nb))
            axes.set_ylim(y_lim_min , y_lim_max)
            axes.grid()

            legend.append('Cluster n°' + str(nb))

        if (-1 in labels) == True:
            spike_data_onelinee = record_spikes_oneline(signal,
                                  fs,
                                  updated_spike_info_all_electrodes.loc[(updated_spike_info_all_electrodes['cluster_label'] == -1) & (updated_spike_info_all_electrodes['Electrode num'] == electrode)],
                                  align_method,
                                  t_before = t_before,
                                  t_after = t_after)
            spike_data_onelinee = np.resize(spike_data_onelinee.values,len(spike_data_onelinee.values))
            spike_data_clusterized_oneline.append(spike_data_onelinee)


            axes = figure.add_subplot(nb_line, 2, nb+2)
            axes.plot(signal.index, signal.values)
            axes.plot(signal.index, spike_data_oneline)
            axes.set_xlabel('Time in ms')
            axes.set_title('Cluster de bruit')
            axes.set_ylim(y_lim_min , y_lim_max)
            axes.grid()

            legend.append('Cluster de bruit')

            axes = figure.add_subplot(nb_line, 2, nb+3)
        else:
            axes = figure.add_subplot(nb_line, 2, nb+2)

        axes.plot(signal.index, signal.values)
        axes.plot(signal.index, np.transpose(spike_data_clusterized_oneline))
        axes.legend(legend)
        axes.set_xlabel('Time in ms')
        axes.set_title('Tous les clusters')
        axes.set_ylim(y_lim_min , y_lim_max)
        axes.grid()
        
        
        
        

#print_spikes_clusterized_oneline(signal,
#                                 updated_spike_infos,
#                                 align_method = 'indice_zero_central',
#                                 t_before = 0.001,
#                                 t_after = 0.002,
#                                 fs = 25000,)

In [115]:
## Fonction qui affiche les spikes des différents clusters sur échelle temporelle

def print_spikes_clusterized_oneline_total(filtereddf,
                                     updated_spike_info_all_electrodes,
                                     align_method = 'indice_zero_central',
                                     t_before = 0.001,
                                     t_after = 0.002,
                                     fs = 25000,
                                     y_lim_min = -50,
                                     y_lim_max = 60,
                                     separate_plot = False):

    electrodes = updated_spike_info_all_electrodes['Electrode num'].values
    nb_electrodes = max(electrodes) + 1
    
    for electrode in range(nb_electrodes):
        
        signal = filtereddf.iloc[:,electrode]
        labels = updated_spike_info_all_electrodes['cluster_label'].values
        nb_clusters = max(labels) + 1

        if (-1 in labels) == True:
            nb_clusters_ = nb_clusters + 2
        else:
            nb_clusters_ = nb_clusters + 1

        nb_line = nb_clusters_//2
        if nb_clusters_%2 != 0:
            nb_line += 1

        ghost_array = np.array(['NaN' for x in range(len(signal))])
        ghost_array = ghost_array.astype(float)
        spike_data_clusterized_oneline = []
        legend = ['Signald\'origine']

        figure = plt.figure()

        plt.gcf().subplots_adjust(left = 0.1, bottom = 0.1, right = 0.9, top = 0.9, wspace = 0.2, hspace = 0.5)
        
        for nb in range(nb_clusters):

            spike_data_onelinee = record_spikes_oneline(signal,
                                  fs,
                                  updated_spike_info_all_electrodes.loc[(updated_spike_info_all_electrodes['cluster_label'] == nb) & (updated_spike_info_all_electrodes['Electrode num'] == electrode)],
                                  align_method,
                                  t_before = t_before,
                                  t_after = t_after)
            spike_data_onelinee = np.resize(spike_data_onelinee.values,len(spike_data_onelinee.values))
            spike_data_clusterized_oneline.append(spike_data_onelinee)


            legend.append('Cluster n°' + str(nb))


        if (-1 in labels) == True:
            spike_data_onelinee = record_spikes_oneline(signal,
                                  fs,
                                  updated_spike_info_all_electrodes.loc[(updated_spike_info_all_electrodes['cluster_label'] == -1) & (updated_spike_info_all_electrodes['Electrode num'] == electrode)],
                                  align_method,
                                  t_before = t_before,
                                  t_after = t_after)
            spike_data_onelinee = np.resize(spike_data_onelinee.values,len(spike_data_onelinee.values))
            spike_data_clusterized_oneline.append(spike_data_onelinee)
            legend.append('Cluster de bruit')

        axes = figure.add_subplot(1, 1, 1)
        axes.plot(signal.index, signal.values)
        axes.plot(signal.index, np.transpose(spike_data_clusterized_oneline))
        axes.legend(legend)
        axes.set_xlabel('Time in ms')
        axes.set_title('Tous les clusters pour: '+str(filtereddf.columns[electrode]))
        axes.set_ylim(y_lim_min , y_lim_max)
        axes.grid()
        
        
        
        

#print_spikes_clusterized_oneline_total(signal,
#                                 updated_spike_infos,
#                                 align_method = 'indice_zero_central',
#                                 t_before = 0.001,
#                                 t_after = 0.002,
#                                 fs = 25000,)

## Histogram of Amplitudes and Time

In [116]:
## Histogramme de différences d'amplitude Peak-to-Peak des spikes

def histogram_spikes_amplitude(spike_info_all_electrodes):
    plt.figure()
    plt.hist(spike_info_all_electrodes['Delta_amplitudes'], bins='auto')
    plt.title("Histogram of Spike Peak-to-Peak Amplitude")
    plt.show()

In [117]:
## Histogramme de l'écartement temporel des spikes (imax-imin)

def histogram_spikes_time(spike_info_all_electrodes):
    plt.figure()
    plt.hist(spike_info_all_electrodes['i_max-i_min'], bins='auto')
    plt.title("Histogram of Spike i_max-i_min")
    plt.show()

In [118]:
## Histogramme des différences d'amplitude Peak-to-Peak des spikes:

## ¡¡¡ATTENTION!!! Il faut que le spike info en entrée, soit mis à jour avec les labels des clusters

def histogram_amplitude_clusterized(updated_spike_info_all_electrodes):
    
    labels = updated_spike_info_all_electrodes['cluster_label'].values
    nb_clusters = max(labels) + 1
    
    for nb in range(nb_clusters):
        local_info = updated_spike_info_all_electrodes.loc[updated_spike_info_all_electrodes['cluster_label'] == nb]
        plt.figure()
        plt.hist(local_info['Delta_amplitudes'], bins='auto')
        plt.title("Histogram of Spike peak-to-peak of cluster n°"+str(nb))
        plt.show()
    
    if(-1 in labels) == True:
        local_info = updated_spike_info_all_electrodes.loc[updated_spike_info_all_electrodes['cluster_label']==-1]
        plt.figure()
        plt.hist(local_info['Delta_amplitudes'], bins='auto')
        plt.title("Histogram of Spike peak-to-peak of cluster of noise")
        plt.show()

In [119]:
## Histogramme des différences d'amplitude Peak-to-Peak des spikes:

## ¡¡¡ATTENTION!!! Il faut que le spike info en entrée, soit mis à jour avec les labels des clusters

def histogram_amplitude_clusterized_superposed(updated_spike_info_all_electrodes):
    
    labels = updated_spike_info_all_electrodes['cluster_label'].values
    nb_clusters = max(labels) + 1
    
    for nb in range(nb_clusters,0,-1):
        local_info = updated_spike_info_all_electrodes.loc[updated_spike_info_all_electrodes['cluster_label'] == nb]
        plt.hist(local_info['Delta_amplitudes'], bins='auto')
        plt.title("Superposition of histogram of Spike peak-to-peak of clusters")
        plt.show()
    
    if(-1 in labels) == True:
        local_info = updated_spike_info_all_electrodes.loc[updated_spike_info_all_electrodes['cluster_label']==-1]
        plt.hist(local_info['Delta_amplitudes'], bins='auto')
        plt.show()

# Tests des fonctions

In [124]:
# Function: PCA_and_KMEANS_spikes_electrodes_combined(spike_data_all_electrodes, spike_info_all_electrodes, nb_PCA_componefnts=3, nb_clusters=3)

PCA_X, KMEANS, updated_spike_info_all_electrodes = PCA_and_KMEANS_spikes_electrodes_combined(spike_data_norm_all_electrodes, spike_info_all_electrodes, 5, 6)


In [125]:
# Function: PCA_plot(spike_data, nb_clusters=3)

PCA_plot(PCA_X)

In [126]:
# Function: print_clusters_3d(labels, PCA_X)

print_clusters_3d(KMEANS.labels_, PCA_X)

In [166]:
# Function: print_spikes_clusterized(spike_data_all_electrodes,
#                             labels,
#                             t_before_alignement = 0,
#                             nb_spike = 20,
#                             y_lim_min = -50,
#                             y_lim_max = 60,
#                             fs = 25000)
                        
print_spikes_clusterized(spike_data_norm_all_electrodes,
                             KMEANS.labels_,
                             t_before_alignement = time_before,
                             nb_spike = 20,
                             y_lim_min = -0.5,
                             y_lim_max = 0.5,
                             fs = fs)                        

In [42]:
# Function: print_spikes_clusterized_oneline(filtereddf,
#                                     updated_spike_info_all_electrodes,
#                                     align_method = 'indice_zero_central',
#                                     t_before = 0.001,
#                                     t_after = 0.002,
#                                     fs = 25000,
#                                     y_lim_min = -50,
#                                     y_lim_max = 60,
#                                     separate_plot = False)

print_spikes_clusterized_oneline(filtereddf,
                                 updated_spike_info_all_electrodes,
                                 align_method = method_align,
                                 t_before = time_before,
                                 t_after = time_after,
                                 fs = fs,)

In [43]:
# Function : print_spikes_clusterized_oneline_total(filtereddf,
#                                     updated_spike_info_all_electrodes,
#                                     align_method = 'indice_zero_central',
#                                     t_before = 0.001,
#                                     t_after = 0.002,
#                                     fs = 25000,
#                                     y_lim_min = -50,
#                                     y_lim_max = 60,
#                                     separate_plot = False)

print_spikes_clusterized_oneline_total(filtereddf,
                                 updated_spike_info_all_electrodes,
                                 align_method = method_align,
                                 t_before = time_before,
                                 t_after = time_after,
                                 fs = fs,)

In [44]:
# Function: histogram_spikes_amplitude(spike_info)

histogram_spikes_amplitude(updated_spike_info_all_electrodes)

/opt/anaconda3/lib/python3.7/tkinter/__init__.py:749: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)
/opt/anaconda3/lib/python3.7/tkinter/__init__.py:1705: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  return self.func(*args)


In [ ]:
# Function: histogram_spikes_time(spike_info)

histogram_spikes_time(updated_spike_info_all_electrodes)

In [ ]:
# Function: histogram_amplitude_clusterized(spike_info)

histogram_amplitude_clusterized(updated_spike_info_all_electrodes)

In [ ]:
# Function: histogram_amplitude_clusterized_superposed(updated_spike_info)

histogram_amplitude_clusterized_superposed(updated_spike_info_all_electrodes)

In [65]:
print_pca_explained_variance_ratio(signal, fs, spike_info)

(38, 151)
(38, 151)
(38, 151)
(38, 151)
(38, 151)
(38, 151)
